In [ ]:
# ================================================================================================
# 완전한 운영 환경 시뮬레이션 파이프라인
# 1단계: 5월 20일까지 데이터로 모델 학습 및 성능 평가
# 2단계: 최고 성능 모델 선택 및 저장
# 3단계: 5월 21일~31일 데이터로 예측 수행
# 4단계: 결과 테이블 생성 및 성능 평가
# ================================================================================================

import pandas as pd
import numpy as np
import time
import warnings
from tqdm import tqdm

# 기존 함수들 활용
def complete_production_simulation_pipeline(centers=None, cutoff_date='2025-05-20'):
    """
    완전한 운영 환경 시뮬레이션 파이프라인
    
    Parameters:
    - centers: 센터별 데이터 딕셔너리
    - cutoff_date: 학습/예측 분할 기준일
    
    Returns:
    - final_results_table: 최종 예측 결과 테이블
    - trained_models_info: 학습된 모델 정보
    - performance_summary: 성능 요약
    """
    
    print(f"{'='*80}")
    print(f"완전한 운영 환경 시뮬레이션 파이프라인 시작")
    print(f"학습 기간: ~ {cutoff_date}")
    print(f"예측 기간: {cutoff_date} 이후")
    print(f"{'='*80}")
    
    # 데이터 로드 확인
    if centers is None:
        try:
            centers = {
                "nanji": nanji,
                "jungnang": jungnang,  
                "seonam": seonam,
                "tancheon": tancheon
            }
            print(f"데이터 로드 완료:")
            for name, df in centers.items():
                print(f"  {name}: {len(df)}행")
        except NameError:
            print("데이터가 로드되지 않았습니다. 먼저 데이터를 로드하세요.")
            return None
    
    cutoff = pd.to_datetime(cutoff_date)
    
    # ========================================================================================
    # 1단계: 각 센터별로 5월 20일까지 데이터로 모델 학습 및 성능 평가
    # ========================================================================================
    print(f"\n{'='*60}")
    print(f"1단계: 모델 학습 및 성능 평가 (~ {cutoff_date})")
    print(f"{'='*60}")
    
    all_training_results = []
    best_models_by_center = {}
    
    for center_name, df in centers.items():
        print(f"\n[{center_name.upper()} 센터 처리 중...]")
        
        # 날짜 변환 및 데이터 분할
        df_work = df.copy()
        df_work['날짜'] = pd.to_datetime(df_work['날짜'])
        
        train_data = df_work[df_work['날짜'] <= cutoff].copy()
        future_data = df_work[df_work['날짜'] > cutoff].copy()
        
        print(f"  학습 데이터: {len(train_data)}행")
        print(f"  예측 데이터: {len(future_data)}행")
        
        if len(train_data) < 50:
            print(f"  학습 데이터가 부족합니다. 건너뜁니다.")
            continue
            
        if len(future_data) == 0:
            print(f"  예측할 데이터가 없습니다. 건너뜁니다.")
            continue
        
        # 모델 성능 평가 (기존 함수 활용)
        center_results = comprehensive_evaluation_comparison(center_name, train_data)
        all_training_results.extend(center_results)
        
        # 최고 성능 모델 선택
        center_best_models = select_and_train_best_models(center_name, train_data, center_results)
        if center_best_models:
            best_models_by_center[center_name] = center_best_models
    
    # ========================================================================================
    # 2단계: 최고 성능 모델 선택 결과 요약
    # ========================================================================================
    print(f"\n{'='*60}")
    print(f"2단계: 최고 성능 모델 선택 완료")
    print(f"{'='*60}")
    
    training_results_df = pd.DataFrame(all_training_results)
    
    for center, models in best_models_by_center.items():
        print(f"\n[{center.upper()} 센터 최고 성능 모델]")
        if 'regression' in models:
            reg_info = models['regression']
            print(f"  회귀: {reg_info['model_name']} (R²={reg_info['performance']['r2']:.3f})")
        if 'classification' in models:
            clf_info = models['classification']
            print(f"  분류: {clf_info['model_name']} (F1={clf_info['performance']['macro_f1']:.3f})")
    
    # ========================================================================================
    # 3단계: 5월 21일~31일 데이터로 예측 수행
    # ========================================================================================
    print(f"\n{'='*60}")
    print(f"3단계: 새로운 데이터 예측 수행 ({cutoff_date} 이후)")
    print(f"{'='*60}")
    
    all_predictions = []
    
    for center_name, df in centers.items():
        if center_name not in best_models_by_center:
            continue
            
        print(f"\n[{center_name.upper()} 센터 예측 중...]")
        
        # 데이터 준비
        df_work = df.copy()
        df_work['날짜'] = pd.to_datetime(df_work['날짜'])
        future_data = df_work[df_work['날짜'] > cutoff].copy()
        
        if len(future_data) == 0:
            continue
        
        # 예측 수행
        center_predictions = make_predictions_for_center(
            center_name, future_data, best_models_by_center[center_name]
        )
        all_predictions.extend(center_predictions)
    
    # ========================================================================================
    # 4단계: 결과 테이블 생성 및 성능 평가
    # ========================================================================================
    print(f"\n{'='*60}")
    print(f"4단계: 결과 테이블 생성 및 성능 평가")
    print(f"{'='*60}")
    
    if not all_predictions:
        print("예측 결과가 없습니다.")
        return None
    
    # 최종 결과 테이블 생성
    final_results_table = create_final_results_table(all_predictions)
    
    # 성능 요약 생성
    performance_summary = create_performance_summary(final_results_table)
    
    # 결과 출력
    print_final_results(final_results_table, performance_summary)
    
    # 결과 저장
    save_final_results(final_results_table, performance_summary, training_results_df)
    
    return final_results_table, best_models_by_center, performance_summary

def select_and_train_best_models(center_name, train_data, evaluation_results):
    """최고 성능 모델 선택 및 학습"""
    
    results_df = pd.DataFrame(evaluation_results)
    successful_results = results_df[results_df['success'] == True]
    
    if len(successful_results) == 0:
        print(f"    성공한 모델이 없습니다.")
        return None
    
    best_models = {}
    
    # 회귀 최고 성능 모델
    reg_results = successful_results[successful_results['type'] == 'regression']
    if len(reg_results) > 0:
        best_reg = reg_results.loc[reg_results['r2'].idxmax()]
        print(f"    최고 회귀 모델: {best_reg['model']} (R²={best_reg['r2']:.3f})")
        
        # 모델 재학습
        reg_pipeline = retrain_best_model(
            train_data, best_reg['model'], 'regression', best_reg['split_method']
        )
        
        if reg_pipeline:
            best_models['regression'] = {
                'model_name': best_reg['model'],
                'pipeline': reg_pipeline['pipeline'],
                'feature_names': reg_pipeline['feature_names'],
                'performance': dict(best_reg),
                'split_method': best_reg['split_method']
            }
    
    # 분류 최고 성능 모델
    clf_results = successful_results[successful_results['type'] == 'classification']
    if len(clf_results) > 0:
        best_clf = clf_results.loc[clf_results['macro_f1'].idxmax()]
        print(f"    최고 분류 모델: {best_clf['model']} (F1={best_clf['macro_f1']:.3f})")
        
        # 모델 재학습
        clf_pipeline = retrain_best_model(
            train_data, best_clf['model'], 'classification', best_clf['split_method']
        )
        
        if clf_pipeline:
            best_models['classification'] = {
                'model_name': best_clf['model'],
                'pipeline': clf_pipeline['pipeline'],
                'feature_names': clf_pipeline['feature_names'],
                'performance': dict(best_clf),
                'split_method': best_clf['split_method']
            }
    
    return best_models if best_models else None

def retrain_best_model(train_data, model_name, model_type, split_method):
    """최고 성능 모델 재학습"""
    
    try:
        target_col = "합계_1일후" if model_type == "regression" else "등급_1일후"
        
        # 전체 학습 데이터로 재학습 (test_size를 매우 작게 설정)
        X_train, X_test, y_train, y_test, feature_names, _, _ = prepare_data_stratified(
            train_data, target_col=target_col, model_type=model_type, 
            test_size=0.05, split_method=split_method
        )
        
        # 전체 데이터 사용 (X_train + X_test)
        X_all = pd.concat([X_train, X_test], ignore_index=True)
        y_all = pd.concat([y_train, y_test], ignore_index=True)
        
        # 모델 구축 및 학습
        if model_type == "regression":
            models = build_regression_models()
        else:
            models = build_classification_models()
        
        model = models[model_name]
        pipeline = make_pipeline_unified(model, model_name, model_type)
        pipeline.fit(X_all, y_all)
        
        return {
            'pipeline': pipeline,
            'feature_names': feature_names
        }
        
    except Exception as e:
        print(f"    모델 재학습 실패 ({model_name}): {e}")
        return None

def make_predictions_for_center(center_name, future_data, trained_models):
    """센터별 예측 수행"""
    
    predictions = []
    
    for task_type, model_info in trained_models.items():
        try:
            pipeline = model_info['pipeline']
            feature_names = model_info['feature_names']
            model_name = model_info['model_name']
            
            # 타겟 컬럼 설정
            target_col = "합계_1일후" if task_type == "regression" else "등급_1일후"
            
            # 예측 데이터 준비
            X_future, y_true = prepare_prediction_data(future_data, feature_names, target_col)
            
            if X_future is None or len(X_future) == 0:
                print(f"    {task_type} 예측 데이터 준비 실패")
                continue
            
            # 예측 수행
            y_pred = pipeline.predict(X_future)
            
            print(f"    {task_type} 예측 완료: {len(y_pred)}개")
            
            # 결과 저장
            for i in range(len(X_future)):
                pred_result = {
                    'date': future_data.iloc[i]['날짜'],
                    'center': center_name,
                    'task_type': task_type,
                    'model_name': model_name,
                    'target_column': target_col,
                    'actual_value': y_true.iloc[i] if i < len(y_true) and not pd.isna(y_true.iloc[i]) else None,
                    'predicted_value': float(y_pred[i])
                }
                predictions.append(pred_result)
                
        except Exception as e:
            print(f"    {task_type} 예측 실패: {e}")
    
    return predictions

def prepare_prediction_data(future_data, expected_features, target_col):
    """예측용 데이터 전처리"""
    
    # 제외할 컬럼들
    not_use_col = [
        '날짜',
        '1처리장','2처리장','정화조','중계펌프장','합계','시설현대화',
        '3처리장','4처리장','합계', '합계_1일후','합계_2일후',
        '등급','등급_1일후','등급_2일후'
    ]
    
    # 피처 데이터 준비
    drop_cols = [c for c in (set(not_use_col) | {target_col}) if c in future_data.columns]
    X_future = future_data.drop(columns=drop_cols, errors="ignore")
    
    # 수치형 변환
    for c in X_future.columns:
        X_future[c] = pd.to_numeric(X_future[c], errors="coerce")
    
    # 실제값 추출
    y_true = None
    if target_col in future_data.columns:
        if target_col == "등급_1일후":
            y_true = pd.to_numeric(future_data[target_col], errors="coerce").astype("Int64")
        else:
            y_true = pd.to_numeric(future_data[target_col], errors="coerce")
    
    # 피처 순서 맞춤 및 누락 피처 처리
    missing_features = set(expected_features) - set(X_future.columns)
    if missing_features:
        for feature in missing_features:
            X_future[feature] = 0
    
    # 피처 순서 맞춤
    X_future = X_future[expected_features].copy()
    
    return X_future, y_true

def create_final_results_table(all_predictions):
    """최종 결과 테이블 생성"""
    
    results_df = pd.DataFrame(all_predictions)
    
    # 날짜 정렬
    results_df = results_df.sort_values(['date', 'center', 'task_type'])
    
    # 평가 지표 계산
    results_df = calculate_prediction_metrics_enhanced(results_df)
    
    return results_df

def calculate_prediction_metrics_enhanced(results_df):
    """향상된 예측 평가 지표 계산"""
    
    results_df = results_df.copy()
    
    # 평가 지표 컬럼 초기화
    results_df['absolute_error'] = None
    results_df['squared_error'] = None
    results_df['percentage_error'] = None
    results_df['correct_prediction'] = None
    results_df['residual'] = None
    
    for idx, row in results_df.iterrows():
        if pd.isna(row['actual_value']) or pd.isna(row['predicted_value']):
            continue
            
        actual = row['actual_value']
        predicted = row['predicted_value']
        
        if row['task_type'] == 'regression':
            # 회귀 평가 지표
            residual = actual - predicted
            abs_error = abs(residual)
            sq_error = residual ** 2
            pct_error = abs(residual) / (abs(actual) + 1e-8) * 100
            
            results_df.at[idx, 'residual'] = residual
            results_df.at[idx, 'absolute_error'] = abs_error
            results_df.at[idx, 'squared_error'] = sq_error
            results_df.at[idx, 'percentage_error'] = pct_error
            
        else:  # classification
            # 분류 평가 지표
            correct = 1 if int(actual) == int(predicted) else 0
            results_df.at[idx, 'correct_prediction'] = correct
    
    return results_df

def create_performance_summary(results_df):
    """성능 요약 생성"""
    
    summary = {}
    
    for center in results_df['center'].unique():
        center_data = results_df[results_df['center'] == center]
        summary[center] = {}
        
        for task_type in ['regression', 'classification']:
            task_data = center_data[center_data['task_type'] == task_type]
            task_data_clean = task_data.dropna(subset=['actual_value', 'predicted_value'])
            
            if len(task_data_clean) > 0:
                if task_type == 'regression':
                    summary[center]['regression'] = {
                        'model_name': task_data_clean.iloc[0]['model_name'],
                        'prediction_count': len(task_data_clean),
                        'mae': task_data_clean['absolute_error'].mean(),
                        'rmse': np.sqrt(task_data_clean['squared_error'].mean()),
                        'mape': task_data_clean['percentage_error'].mean(),
                        'r2_on_predictions': calculate_r2_on_predictions(
                            task_data_clean['actual_value'], 
                            task_data_clean['predicted_value']
                        )
                    }
                else:
                    summary[center]['classification'] = {
                        'model_name': task_data_clean.iloc[0]['model_name'],
                        'prediction_count': len(task_data_clean),
                        'accuracy': task_data_clean['correct_prediction'].mean(),
                        'correct_count': int(task_data_clean['correct_prediction'].sum()),
                        'total_count': len(task_data_clean)
                    }
    
    return summary

def calculate_r2_on_predictions(y_true, y_pred):
    """예측값에 대한 R² 계산"""
    try:
        from sklearn.metrics import r2_score
        return r2_score(y_true, y_pred)
    except:
        return None

def print_final_results(results_df, performance_summary):
    """최종 결과 출력"""
    
    print(f"\n{'='*60}")
    print(f"=== 최종 예측 결과 요약 ===")
    print(f"{'='*60}")
    
    # 전체 요약
    total_predictions = len(results_df)
    date_range = f"{results_df['date'].min()} ~ {results_df['date'].max()}"
    centers = results_df['center'].unique()
    
    print(f"예측 기간: {date_range}")
    print(f"총 예측 건수: {total_predictions}")
    print(f"센터 수: {len(centers)} ({', '.join(centers)})")
    
    # 센터별 성능 요약
    for center, perf in performance_summary.items():
        print(f"\n--- {center.upper()} 센터 성능 ---")
        
        if 'regression' in perf:
            reg = perf['regression']
            print(f"  회귀 ({reg['model_name']}):")
            print(f"    예측 건수: {reg['prediction_count']}")
            print(f"    MAE: {reg['mae']:.2f}")
            print(f"    RMSE: {reg['rmse']:.2f}")
            print(f"    MAPE: {reg['mape']:.1f}%")
            if reg['r2_on_predictions'] is not None:
                print(f"    R²: {reg['r2_on_predictions']:.3f}")
        
        if 'classification' in perf:
            clf = perf['classification']
            print(f"  분류 ({clf['model_name']}):")
            print(f"    예측 건수: {clf['prediction_count']}")
            print(f"    정확도: {clf['accuracy']:.1%}")
            print(f"    정답 개수: {clf['correct_count']}/{clf['total_count']}")
    
    # 결과 테이블 미리보기
    print(f"\n--- 결과 테이블 미리보기 ---")
    display_columns = ['date', 'center', 'task_type', 'model_name', 'actual_value', 'predicted_value']
    if all(col in results_df.columns for col in display_columns):
        print(results_df[display_columns].head(10).to_string(index=False))

def save_final_results(results_df, performance_summary, training_results_df):
    """최종 결과 저장"""
    
    timestamp = time.strftime("%Y%m%d_%H%M%S")
    base_filename = f"production_simulation_{timestamp}"
    
    try:
        # 1. 예측 결과 테이블 저장
        results_filename = f"{base_filename}_predictions.csv"
        results_df.to_csv(results_filename, index=False, encoding='utf-8-sig')
        print(f"\n예측 결과 저장: {results_filename}")
        
        # 2. 성능 요약 저장
        summary_data = []
        for center, perf in performance_summary.items():
            for task_type, metrics in perf.items():
                summary_row = {'center': center, 'task_type': task_type}
                summary_row.update(metrics)
                summary_data.append(summary_row)
        
        if summary_data:
            summary_df = pd.DataFrame(summary_data)
            summary_filename = f"{base_filename}_summary.csv"
            summary_df.to_csv(summary_filename, index=False, encoding='utf-8-sig')
            print(f"성능 요약 저장: {summary_filename}")
        
        # 3. 학습 결과 저장
        if len(training_results_df) > 0:
            training_filename = f"{base_filename}_training.csv"
            training_results_df.to_csv(training_filename, index=False, encoding='utf-8-sig')
            print(f"학습 결과 저장: {training_filename}")
        
        print(f"\n모든 결과가 저장되었습니다. 파일명 접두사: {base_filename}")
        
    except Exception as e:
        print(f"결과 저장 중 오류 발생: {e}")

# ================================================================================================
# 메인 실행 함수
# ================================================================================================

def run_complete_production_pipeline(cutoff_date='2025-05-20'):
    """완전한 파이프라인 실행"""
    
    print("완전한 운영 환경 시뮬레이션 파이프라인을 시작합니다...")
    print("이 과정은 다소 시간이 걸릴 수 있습니다.")
    
    start_time = time.time()
    
    # 파이프라인 실행
    results = complete_production_simulation_pipeline(cutoff_date=cutoff_date)
    
    end_time = time.time()
    elapsed_time = end_time - start_time
    
    print(f"\n{'='*60}")
    print(f"파이프라인 실행 완료!")
    print(f"총 소요시간: {elapsed_time:.1f}초 ({elapsed_time/60:.1f}분)")
    print(f"{'='*60}")
    
    return results

# ================================================================================================
# 사용 예시
# ================================================================================================

if __name__ == "__main__":
    print("=== 완전한 운영 환경 시뮬레이션 파이프라인 ===")
    print()
    print("사용법:")
    print("results = run_complete_production_pipeline()")
    print()
    print("또는 cutoff_date를 변경하여:")
    print("results = run_complete_production_pipeline(cutoff_date='2025-05-15')")
    print()
    print("반환값:")
    print("- results[0]: 최종 예측 결과 테이블")
    print("- results[1]: 학습된 모델 정보")  
    print("- results[2]: 성능 요약")
    print()
    print("생성되는 파일:")
    print("- production_simulation_YYYYMMDD_HHMMSS_predictions.csv")
    print("- production_simulation_YYYYMMDD_HHMMSS_summary.csv")
    print("- production_simulation_YYYYMMDD_HHMMSS_training.csv")